# Associate IAM Policies with EKS Worker Nodes


In [1]:
%%bash

### Source the environment
source ~/.bash_profile

export INSTANCE_ROLE_NAME=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .RoleName")
echo "export INSTANCE_ROLE_NAME=${INSTANCE_ROLE_NAME}" | tee -a ~/.bash_profile

export INSTANCE_PROFILE_ARN=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .Arn")
echo "export INSTANCE_PROFILE_ARN=${INSTANCE_PROFILE_ARN}" | tee -a ~/.bash_profile

export INSTANCE_ROLE_NAME=eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-4DG6CRI38MUD
export INSTANCE_PROFILE_ARN=arn:aws:iam::343441690612:role/eksctl-cluster-nodegroup-cpu-node-NodeInstanceRole-4DG6CRI38MUD


# Attach Roles

In [2]:
%%bash

### Source the environment
source ~/.bash_profile

#### Allow Access from/to the Elastic Container Registry (ECR)
# This allows our cluster worker nodes to load custom Docker images (ie. models) from ECR.  We will load these custom Docker images in a later section.
aws iam attach-role-policy --role-name $INSTANCE_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

echo "Completed"

Completed


# Associated IAM and OIDC
To use IAM Roles for Service Accounts in your cluster, you must create an OIDC identity provider in the IAM console.  

See https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html for more info.

# _This may take a few minutes.  Please be patient._


In [3]:
%%bash

### Source the environment
source ~/.bash_profile

eksctl utils associate-iam-oidc-provider --cluster ${AWS_CLUSTER_NAME} --approve

echo "Completed"

[ℹ]  eksctl version 0.25.0
[ℹ]  using region ap-northeast-2
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 34.327784ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 70.467186ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 193.029524ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 438.904696ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 524.595568ms
[!]  re

In [4]:
%%bash

### Source the environment
source ~/.bash_profile

aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.identity.oidc.issuer" --output text

echo "Completed"

https://oidc.eks.ap-northeast-2.amazonaws.com/id/89C372954BDCA6AC1A2F60DCFEF39EAD
Completed


# Update `~/.kube/config` with our new EKS cluster

In [5]:
%%bash

source ~/.bash_profile

aws eks --region ${AWS_REGION} update-kubeconfig --name ${AWS_CLUSTER_NAME} 

Added new context arn:aws:eks:ap-northeast-2:343441690612:cluster/cluster to /home/ec2-user/.kube/config


# Verify Your EKS Cluster


In [6]:
%%bash

kubectl get ns


NAME              STATUS   AGE
default           Active   17m
kube-node-lease   Active   17m
kube-public       Active   17m
kube-system       Active   17m


In [7]:
%%bash

kubectl get nodes


NAME                                                STATUS   ROLES    AGE   VERSION
ip-192-168-39-223.ap-northeast-2.compute.internal   Ready    <none>   12m   v1.17.9-eks-4c6976
ip-192-168-74-46.ap-northeast-2.compute.internal    Ready    <none>   12m   v1.17.9-eks-4c6976


In [8]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>